# Learning the 7-mer production fitness landscape


# Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import Image

from scipy.stats import gaussian_kde

from utils_f4f import CustomEarlyStopping, AA_hotencoding, parent_model

In [ ]:
mpl.rcParams['svg.fonttype'] = 'none'
mpl.rcParams['font.family'] = ['sans-serif']
mpl.rcParams['font.sans-serif'] = ['Arial']
mpl.rcParams['text.usetex'] = False
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42

------

# Production Fitness Modeling


In [ ]:
# Data

# Modeling data file 
FileName = 'data/modeling_library_production_fitness.csv';
df_modeling = pd.read_csv(FileName, usecols=['AA', 'Label','Production'])


# Independent testing data file 
FileName = 'data/assessment_library_production_fitness.csv';
df_independentTest = pd.read_csv(FileName, usecols=['AA', 'Label','Production'])


In [ ]:
# Preprocessing

# log2Enr
df_modeling['Output'] = np.log2(df_modeling.Production)
df_independentTest['Output'] = np.log2(df_independentTest.Production) 

# Remove non-detected
df_missing = df_modeling[np.isinf(df_modeling.Output) & (df_modeling.Label =='Designed')]

df_modeling = df_modeling[~np.isnan(df_modeling.Output) & ~np.isinf(df_modeling.Output)]
df_modeling.reset_index(drop = True, inplace=True)

df_independentTest = df_independentTest[~np.isnan(df_independentTest.Output) & ~np.isinf(df_independentTest.Output)]
df_independentTest.reset_index(drop = True, inplace=True)


# Slice out the uniform designed sets for modeling and assessment 
df_modeling = df_modeling[df_modeling.Label =='Designed']
df_modeling.reset_index(drop = True, inplace=True)

df_independentTest = df_independentTest[df_independentTest.Label =='Designed']
df_independentTest.reset_index(drop = True, inplace=True)



In [ ]:
# Fitness landscape

print(df_modeling.Output)
n, bins, patches = plt.hist(x=df_modeling.Output, bins='auto',
                            alpha=0.7, rwidth=0.85)
plt.grid(axis='y', alpha=0.75)
plt.xlabel('Measured Production Fitness')
plt.ylabel('Frequency')
plt.title('Fitness Landscape | 7mer insertion at 588, AAV9')
plt.show()


In [ ]:
# Train-Test split

train_size = 24000
validation_size = 5000
train, validate, test = np.split(df_modeling.sample(frac=1), 
                                 [train_size, train_size+validation_size])

In [ ]:
# Hot Encoding

AA_col = 'AA'

train_x =  np.asarray([AA_hotencoding(variant) for variant in train[AA_col]])
train_y = train.Output

validate_x = np.asarray([AA_hotencoding(variant) for variant in validate[AA_col]])
validate_y = validate.Output

test_x = np.asarray([AA_hotencoding(variant) for variant in test[AA_col]])
test_y = test.Output

missing_x = np.asarray([AA_hotencoding(variant) for variant in df_missing[AA_col]])

independent_test_x = np.asarray([AA_hotencoding(variant) for variant in df_independentTest[AA_col]])
independent_test_y = df_independentTest.Output


print(test_x[0])

In [ ]:
# Training

# Define model 
model = parent_model()

# Training parameters 
batch_size = 500
EpochCount = 150
#EpochCount = 5 #Used for quick excusion of the Notebook; please use a higher number instead to allow proper convergence


# Fit model 
model.fit(train_x, train_y, batch_size=batch_size, epochs=EpochCount, 
          validation_data=(validate_x, validate_y),verbose=2,
              callbacks=[CustomEarlyStopping(ratio=0.90, patience=3, restore_best_weights = True)])


# Save model
ModelFileName = 'fit4function_models/fit4function_model_production_fitness_588_7mer'
model.save(ModelFileName+'.h5')

------------

# Internal validation


In [ ]:
# Data

# Measured
x = np.array(test_y)

# Predict for test variants
y = model.predict(test_x)
y = np.reshape(y, (1,y.shape[0]))[0]

# Predict for non-detected variants 
y_missing = model.predict(missing_x)
y_missing = np.reshape(y_missing, (y_missing.shape[0], 1))


In [ ]:
# Figure

# Kernel density estimate
kernel = gaussian_kde(np.vstack([x, y]))
c = kernel(np.vstack([x, y])) # this takes a while


# Figure Configurations 
sns.set_theme(style='ticks', font_scale=0.75, rc={
    'svg.fonttype': 'none',
    'font.sans-serif': ['Arial'],
    'font.family': 'sans-serif',
    'text.usetex': False,
    'pdf.fonttype': 42,
    'ps.fonttype': 42,
    'font.size': 9,
    'axes.labelsize': 9,
    'axes.titlesize': 9,
    'lines.linewidth': 0.5,
    'axes.linewidth': 0.5,
    'legend.fontsize': 9,
    'legend.title_fontsize': 9,
    'xtick.major.size': 3,
    'xtick.major.pad': 3,
    'xtick.major.width': 0.5,
    'ytick.major.size': 3,
    'ytick.major.pad': 3,
    'ytick.major.width': 0.5,
})

pt_size = 0.2

fig = plt.figure(figsize=(1.4, 1.3), dpi=300)
gs = fig.add_gridspec(
    1, 2, 
    width_ratios=[1, 6], 
    hspace=0, wspace=0,
    left=0.3, right=0.93, bottom=0.25, top=0.9
)
gs2 = fig.add_gridspec(
    1, 1,
    left=0.94, right=0.97, bottom=0.25, top=0.9
)


# Axis limits
xlim = [-13.5, 9.5]
xticks = [-10, -5, 0, 5]
yticks = [-10, -5, 0, 5]

cmap = mpl.cm.inferno

# BOTH
ax = fig.add_subplot(gs[0, 1])
ax.scatter(
    x, y, c=c, s=pt_size, cmap=cmap, 
    rasterized=True, linewidth=0, edgecolors=None
)
ax.set_xlim(xlim); ax.set_ylim(xlim)
ax.set_xticks(xticks); ax.set_yticks(yticks)
ax.set_yticklabels([])
ax.set_xlabel('Measured', labelpad=2)
ax.text(0, 0, s='Missing', transform=ax.transAxes, rotation=20, ha='right', va='top', color='r')

# Correlation text
ax.text(
    x=0.05, y=0.95, s=r'$r$ = {:.3f}'.format(np.corrcoef(x, y)[0, 1]), 
    ha='left', va='top', transform=ax.transAxes, fontsize=7
)

# Missing from X
ax = fig.add_subplot(gs[0, 0])
ax.hist(y_missing, bins=bins, edgecolor='none', orientation='horizontal', density=True, color='r')
ax.set_ylim(xlim)
ax.set_xticks([]); ax.set_yticks([-10, -5, 0, 5])
ax.text(1.0, 0.97, 'n = {}'.format(len(y_missing)), 
        transform=ax.transAxes, ha='right', va='top', fontsize=7, rotation=90, color='r')
ax.set_ylabel('Predicted', labelpad=1)
ax.tick_params(axis='both', labelsize=7, length=2, pad=1)

# Colorbar
ax = fig.add_subplot(gs2[0, 0])
norm = mpl.colors.Normalize(vmin=np.min(c), vmax=np.max(c))
cb = mpl.colorbar.ColorbarBase(
    ax, 
    cmap=cmap,
    norm=norm,
    orientation='vertical'
)
cb.set_ticks([])
ax.text(1, 1.02, 'Density', transform=ax.transAxes, ha='right', va='bottom')

filename = 'figures/fig2e_production_internal_test'
fig.savefig('{}.png'.format(filename))
fig.savefig('{}_600dpi.svg'.format(filename),dpi=600)
fig.savefig('{}_1200dpi.svg'.format(filename),dpi=1200)

plt.close()

Image(filename + '.png')

In [ ]:
# save results 
df_results = pd.DataFrame({'Measured': x, 'Predicted': y})
df_results.to_csv('results/production_internal_testing.csv')

-------
# Independent validation



In [ ]:
# Data 

# Measured
x = np.array(independent_test_y)

# Predicted
y = model.predict(independent_test_x)
y = np.reshape(y, (1, y.shape[0]))[0]



In [ ]:
# Figure

# Kernel density estimate
kernel = gaussian_kde(np.vstack([x, y]))
c = kernel(np.vstack([x, y])) # this takes a while


# Figure configurations 
sns.set_theme(style='ticks', font_scale=0.75, rc={
    'svg.fonttype': 'none',
    'font.sans-serif': ['Arial'],
    'font.family': 'sans-serif',
    'text.usetex': False,
    'pdf.fonttype': 42,
    'ps.fonttype': 42,
    'font.size': 9,
    'axes.labelsize': 9,
    'axes.titlesize': 9,
    'lines.linewidth': 0.5,
    'axes.linewidth': 0.5,
    'legend.fontsize': 9,
    'legend.title_fontsize': 9,
    'xtick.major.size': 3,
    'xtick.major.pad': 3,
    'xtick.major.width': 0.5,
    'ytick.major.size': 3,
    'ytick.major.pad': 3,
    'ytick.major.width': 0.5,
})

pt_size = 0.2

cmap = mpl.cm.inferno

# Axis limits
xlim = [-13.5, 9.5]
xticks = [-10, -5, 0, 5]

fig = plt.figure(figsize=(1.3, 1.3), dpi=300)
gs = fig.add_gridspec(
    1, 1, left=0.26, right=0.9, bottom=0.21, top=0.88
)
gs2 = fig.add_gridspec(
    1, 1, left=0.91, right=0.94, bottom=0.21, top=0.88
)

# BOTH
ax = fig.add_subplot(gs[0, 0])
ax.scatter(
    x, y, c=c, s=pt_size, cmap=cmap, 
    rasterized=True, edgecolors=None, linewidth=0
)
ax.set_xlim(xlim); ax.set_ylim(xlim)
ax.set_xticks(xticks); ax.set_yticks(xticks)
ax.set_xlabel('Measured', labelpad=1)
ax.set_ylabel('Predicted', labelpad=0)


# Correlation text
ax.text(
    x=0.05, y=0.95, s=r'$\rho$ = {:.3f}'.format(np.corrcoef(x, y)[0, 1]), 
    transform=ax.transAxes, ha='left', va='top', fontsize=7
)

ax.tick_params(axis='x', labelsize=7, length=2, pad=2)
ax.tick_params(axis='y', labelsize=7, length=2, pad=1)


# Colorbar
ax = fig.add_subplot(gs2[0, 0])
norm = mpl.colors.Normalize(vmin=np.min(c), vmax=np.max(c))
cb = mpl.colorbar.ColorbarBase(
    ax, 
    cmap=cmap,
    norm=norm,
    orientation='vertical'
)
cb.set_ticks([])
ax.text(1, 1.02, 'Density', transform=ax.transAxes, ha='right', va='bottom')

filename = 'figures/fig2f_production_independent_test'
fig.savefig('{}.png'.format(filename))
fig.savefig('{}_600dpi.svg'.format(filename),dpi=600)
fig.savefig('{}_1200dpi.svg'.format(filename),dpi=1200)

plt.close()

Image(filename + '.png')

In [ ]:
# save results 
df_results = pd.DataFrame({'Measured': x, 'Predicted': y})
df_results.to_csv('results/production_independent_testing.csv')

------
# Effect of training size variation on modeling 

Performance of the production fitness prediction model across different training set sizes (n = 10 models, mean ± s.d.). 

In [ ]:
# Train model over different training sizes 

# Initial Train-Test split
train_size = 50000
validation_size = 5000
train, validate, test = np.split(df_modeling.sample(frac=1), 
                                 [train_size, train_size+validation_size])

# Parameters 
L1 = 140
L2 = 20
batch_size = 500
#EpochCount = 150
EpochCount = 5 #Used for quick excusion of the Notebook; please use a higher number instead to allow proper convergence



# Initialization 
df = pd.DataFrame()
df['Train Size'] = np.array(range(1000,31000,1000))
meanPerformanceR = [] 
stdR = []; 

# Iteration 
for crnt_TrainSize in df['Train Size']:
    crnt_TrainSize_R = [] 
    print('Crnt training size = ', str(crnt_TrainSize))
    for ii in range(10): 
        print(str(ii))
        
        # Slice input 
        train_slice = train.loc[np.random.choice(train.index, crnt_TrainSize, replace=False)]
        train_slice_x =  np.asarray([AA_hotencoding(variant) for variant in train_slice[AA_col]])
        train_slice_y = train_slice.Output

        # Define and train model 
        model = parent_model()
        model.fit(train_slice_x, train_slice_y, batch_size=batch_size, epochs=EpochCount, 
                  validation_data=(validate_x, validate_y),verbose=0,
              callbacks=[CustomEarlyStopping(ratio=0.90, patience=3, restore_best_weights = True)])

        # Record performance 
        y = model.predict(test_x, verbose=0)
        y = np.reshape(y, (1, y.shape[0]))[0]
        
        crnt_r = np.corrcoef(np.array(test_y),y)[0, 1]
        crnt_TrainSize_R.append(crnt_r)
    
    # Average performance per train size 
    meanPerformanceR.append(np.mean(crnt_TrainSize_R))
    stdR.append(np.std(crnt_TrainSize_R))

# Summary Stats 
df['mean Performance R'] = np.array(meanPerformanceR)
df['std R'] = np.array(stdR)

print(df)
   

In [ ]:
# save results 
df.to_csv('results/production_train_size_variation.csv')

In [ ]:
# Figure Configurations
sns.set_theme(style='ticks', font_scale=0.75, rc={
    'svg.fonttype': 'none',
    'font.sans-serif': ['Arial'],
    'font.family': 'sans-serif',
    'text.usetex': False,
    'pdf.fonttype': 42,
    'ps.fonttype': 42,
    'font.size': 7,
    'axes.labelsize': 9,
    'axes.titlesize': 9,
    'lines.linewidth': 0.5,
    'axes.linewidth': 0.5,
    'legend.fontsize': 9,
    'legend.title_fontsize': 9,
    'xtick.major.size': 3,
    'xtick.major.pad': 3,
    'xtick.major.width': 0.5,
    'ytick.major.size': 3,
    'ytick.major.pad': 3,
    'ytick.major.width': 0.5,
})

fig = plt.figure(figsize=(1.3, 1.3), dpi=300)
gs = fig.add_gridspec(1, 1, bottom=0.25, top=0.85, left=0.35, right=0.9)

ax = fig.add_subplot(gs[0, 0])

# Error bars 
ax.errorbar(df['Train Size'], df['mean Performance R'], yerr=df['std R'], 
            rasterized=True, linewidth=0.75, fmt='-ob',
           markersize=1.0, markerfacecolor='none', markeredgecolor='b', markeredgewidth=0.5,
           capsize=1.0, ecolor='#888', elinewidth=0.5, capthick=0.5, barsabove=True)

# Axis limits, ticks, and labels 
ax.set_xlim([200, 30000])
ax.set_xticks([1000, 10000, 20000, 30000])
ax.set_xticklabels(['1K', '10K', '20K', '30K'])
ax.set_yticks([0,  0.2,  0.4,  0.6,  0.8,  1.0])
ax.set_xlabel('Train Size', labelpad=2)
ax.set_ylabel('Test Performance (R)  ', labelpad=2)


# Save and display figure 
filename = 'figures/fig2g_production_train_size_variation'
fig.savefig('{}.png'.format(filename))
fig.savefig('{}_600dpi.svg'.format(filename),dpi=600)
fig.savefig('{}_1200dpi.svg'.format(filename),dpi=1200)

plt.close()

Image(filename + '.png')

Note: 
A small epoch count (EpochCount = 5) was used for quick CPU excusion of the Notebook in training the 30X10 models; please use a higher epoch count instead to allow proper model convergence and obtain results similar to Fig. 2g of the Fit4Function manuscript. 
